# Data generation: using python to sweep over methods and parameters

In this notebook, we illustrate how to use python to generate and run a list of commands. In this example, we generate a list of QIIME 1.9.0 ``assign_taxonomy.py`` commands, though this workflow for command generation is generally very useful for performing *parameter sweeps* (i.e., exploration of sets of parameters for achieving a specific result for comparative purposes). 

## Environment preparation

In [1]:
from os.path import join, expandvars 
from joblib import Parallel, delayed
from glob import glob
from os import system
from tax_credit.framework_functions import (parameter_sweep,
                                            generate_per_method_biom_tables,
                                            move_results_to_repository)


In [2]:
project_dir = expandvars("$HOME/Desktop/projects/tax-credit")
analysis_name= "mock-community"
data_dir = join(project_dir, "data", analysis_name)

reference_database_dir = expandvars("$HOME/Desktop/ref_dbs/")
results_dir = expandvars("$HOME/Desktop/projects/mock-community/")

## Preparing data set sweep

First, we're going to define the data sets that we'll sweep over. The following cell does not need to be modified unless if you wish to change the datasets or reference databases used in the sweep.

In [3]:
dataset_reference_combinations = [
 ('mock-1', 'gg_13_8_otus'), # formerly S16S-1
 ('mock-2', 'gg_13_8_otus'), # formerly S16S-2
 ('mock-3', 'gg_13_8_otus'), # formerly Broad-1
 ('mock-4', 'gg_13_8_otus'), # formerly Broad-2
 ('mock-5', 'gg_13_8_otus'), # formerly Broad-3
 ('mock-6', 'gg_13_8_otus'), # formerly Turnbaugh-1
 ('mock-7', 'gg_13_8_otus'), # formerly Turnbaugh-2
 ('mock-8', 'gg_13_8_otus'), # formerly Turnbaugh-3
 ('mock-9', 'unite_20.11.2016_clean_fullITS'), # formerly ITS1
 ('mock-10', 'unite_20.11.2016_clean_fullITS'), # formerly ITS2-SAG
 ('mock-12', 'gg_13_8_otus'), # Extreme
 ('mock-13', 'gg_13_8_otus_full16S'), # kozich-1
 ('mock-14', 'gg_13_8_otus_full16S'), # kozich-2
 ('mock-15', 'gg_13_8_otus_full16S'), # kozich-3
 ('mock-16', 'gg_13_8_otus'), # schirmer-1
 ('mock-18', 'gg_13_8_otus'),
 ('mock-19', 'gg_13_8_otus'),
 ('mock-20', 'gg_13_8_otus'),
 ('mock-21', 'gg_13_8_otus'),
 ('mock-22', 'gg_13_8_otus'),
 ('mock-23', 'gg_13_8_otus'),
 ('mock-24', 'unite_20.11.2016_clean_fullITS'),
 ('mock-25', 'unite_20.11.2016_clean_fullITS'),
 ('mock-26-ITS1', 'unite_20.11.2016_clean_fullITS'),
 ('mock-26-ITS9', 'unite_20.11.2016_clean_fullITS'),
]

reference_dbs = {'gg_13_8_otus' : (join(reference_database_dir, 'gg_13_8_otus/rep_set/99_otus/dna-sequences.fasta'), 
                                   join(reference_database_dir, 'gg_13_8_otus/taxonomy/99_otu_taxonomy.txt')),
                 'gg_13_8_otus_full16S' : (join(reference_database_dir, 'gg_13_8_otus/rep_set/99_otus.fasta'), 
                                   join(reference_database_dir, 'gg_13_8_otus/taxonomy/99_otu_taxonomy.txt')),
                 'unite_20.11.2016_clean_fullITS' : (join(reference_database_dir, 'sh_qiime_release_20.11.2016/developer/sh_refs_qiime_ver7_99_20.11.2016_dev_clean.fasta'), 
                                       join(reference_database_dir, 'sh_qiime_release_20.11.2016/developer/sh_taxonomy_qiime_ver7_99_20.11.2016_dev_clean.tsv')),
                 'unite_20.11.2016' : (join(reference_database_dir, 'sh_qiime_release_20.11.2016/developer/sh_refs_qiime_ver7_99_20.11.2016_dev_ITS1Ff-ITS2r_trim250/dna-sequences.fasta'), 
                                       join(reference_database_dir, 'sh_qiime_release_20.11.2016/developer/sh_taxonomy_qiime_ver7_99_20.11.2016_dev.txt'))}

## Preparing the method/parameter combinations and generating commands

Now we set the methods and method-specific parameters that we want to sweep. Modify to sweep other methods. Note how method_parameters_combinations feeds method/parameter combinations to parameter_sweep() in the cell below.

### Assignment Using QIIME 1 or Command-Line Classifiers

Here we provide an example of taxonomy assignment using legacy ``QIIME 1`` classifiers executed on the command line. To accomplish this, we must first convert ``commands`` to a string, which we then pass to bash for execution. As ``QIIME 1`` is written in python-2, we must also activate a separate environment in which QIIME 1 [has been installed](http://qiime.org/install/install.html). If any environmental variables need to be set (in this example, the [RDP_JAR_PATH](http://qiime.org/install/alternative.html#rdp-jar-path-environment-variable)), we must also source the .bashrc file.

In [4]:
method_parameters_combinations = { # probabalistic classifiers
              'rdp': {'confidence': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                     0.6, 0.7, 0.8, 0.9, 1.0]},
              
              # global alignment classifiers
              'uclust': {'min_consensus_fraction': [0.51, 0.76, 1.0], 
                         'similarity': [0.9, 0.97, 0.99],
                         'uclust_max_accepts': [1, 3, 5]},
             
              # local alignment classifiers
              'sortmerna': {'sortmerna_e_value': [1.0],
                            'min_consensus_fraction': [0.51, 1.0], 
                            'similarity': [0.9, 0.99],
                            'sortmerna_best_N_alignments ': [1, 5],
                            'sortmerna_coverage' : [0.9]},
              'blast' : {'blast_e_value' : [0.0000000001, 1000]}
             }

Now enter the template of the command to sweep, and generate a list of commands with ``parameter_sweep()``.

Fields must adhere to following format:

                      {0} = output directory
                      {1} = input data
                      {2} = reference sequences
                      {3} = reference taxonomy
                      {4} = method name
                      {5} = other parameters

In [5]:
command_template = "source activate qiime1; source ~/.bashrc; mkdir -p {0} ; assign_taxonomy.py -v -i {1} -o {0} -r {2} -t {3} -m {4} {5} --rdp_max_memory 8000"
        
commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='rep_seqs.fna', output_name='rep_seqs_tax_assignments.txt')


As a sanity check, we can look at the first command that was generated and the number of commands generated.

In [6]:
print(len(commands))
commands[0]

96


'source activate qiime1; source ~/.bashrc; mkdir -p /Users/nbokulich/Desktop/projects/mock-community/mock-26-ITS1/unite_20.11.2016_clean_fullITS/uclust/0.51:0.9:1 ; assign_taxonomy.py -v -i /Users/nbokulich/Desktop/projects/tax-credit/data/mock-community/mock-26-ITS1/rep_seqs.fna -o /Users/nbokulich/Desktop/projects/mock-community/mock-26-ITS1/unite_20.11.2016_clean_fullITS/uclust/0.51:0.9:1 -r /Users/nbokulich/Desktop/ref_dbs/sh_qiime_release_20.11.2016/developer/sh_refs_qiime_ver7_99_20.11.2016_dev_clean.fasta -t /Users/nbokulich/Desktop/ref_dbs/sh_qiime_release_20.11.2016/developer/sh_taxonomy_qiime_ver7_99_20.11.2016_dev_clean.tsv -m uclust --similarity 0.9 --min_consensus_fraction 0.51 --uclust_max_accepts 1 --rdp_max_memory 8000'

Finally, we run our commands.

In [7]:
Parallel(n_jobs=1)(delayed(system)(command) for command in commands)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## Generate per-method biom tables

Modify the taxonomy_glob below to point to the taxonomy assignments that were generated above. This may be necessary if filepaths were altered in the preceding cells.

In [8]:
taxonomy_glob = join(results_dir, '*', '*', '*', '*', 'rep_seqs_tax_assignments.txt')
generate_per_method_biom_tables(taxonomy_glob, data_dir)

## Move result files to repository

Add results to the tax-credit directory (e.g., to push these results to the repository or compare with other precomputed results in downstream analysis steps). The precomputed_results_dir path and methods_dirs glob below should not need to be changed unless if substantial changes were made to filepaths in the preceding cells.

In [9]:
precomputed_results_dir = join(project_dir, "data", "precomputed-results", analysis_name)
method_dirs = glob(join(results_dir, '*', '*', '*', '*'))
move_results_to_repository(method_dirs, precomputed_results_dir)